- Linear regression using Gradient descent

#### Gradient Descent

- It is a first order methord for solving an optimization problem
- Gradient Descent with a step size $t_k$ at the $k^{th}$ iteration
    - $ x ^ {k+1} = x^{k} - t_k \nabla f(x^k)$
    - Here $\nabla f(x^k)$ is the gradient at point $x^k$
- This method is commonly used in Deep learning and machine learning to minimize cost functions
- For convex optimization it gives global optimum under failry general conditions
- for non-convex optimization, it may achieve a local optimum

<b> In relation to Linear regression we are trying to minimize the squared errors </b>
- $min f(b) = || y-Xb ||^{2}$
    - $X \in R^{n\times(p+1)}$
- $\nabla f(b) = 2X^{T}(Xb-y)$
- $b^{k+1} = b^{k} - 2t_{k} X^{T}(Xb^{k}-y)$

<b> Some of the stopping criterion could be the following </b>
- $||\nabla f(x)||_{2}$ is sufficiently small
- $||x^{k+1}-x^{k}||_{2}$ or $|f(x^{k+1})-f(x^{k})|$ is small

<b> Learning Rate in a gradient Descent</b>
<br>![Alt Text](https://builtin.com/sites/www.builtin.com/files/styles/ckeditor_optimize/public/inline-images/national/gradient-descent-learning-rate.png)</br>

[Link for More on hyperparameters](https://developers.google.com/machine-learning/crash-course/linear-regression/hyperparameters)

Some types of Gradient Descent
- Batch Gradient Descent
- Stochastic Gradient Descent
- Mini batch Gradient Descent


[Link to diff between batch and SGD](https://www.geeksforgeeks.org/difference-between-batch-gradient-descent-and-stochastic-gradient-descent/#:~:text=background%2Dcolor%3A%20%23ffffff%3B%20%7D-,Batch%20Gradient%20Descent,can%20help%20reduce%20overfitting%20by%20updating%20the%20model%20parameters%20more%20frequently.,-Comment)